In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 1
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 1
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]
unique_signals = [1,2,3,4]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'

In [ ]:
adf = pd.read_csv(path_results+'preanalyzed.csv')  
df_diff = pd.read_csv(path_results+'per_difficulty.csv')  
dfANOVA = pd.read_csv(path_results+'forANOVA.csv') 
dfLR = pd.read_csv(path_results+'linearRegr_diff.csv') 

adf = pd.merge(dfLR,adf,on='user_sessionID',how='outer') 
# sort df by userID and sessionID in second place
adf = adf.sort_values(by = ['userID', 'sessionID_y'])

userids = adf['userID'].unique()
userids = sorted(userids)
nsub = len(userids)
sessionids = adf['sessionID_x'].unique()
sessionids = sorted(sessionids)

# morning df
mdf = adf[adf['sessionID_x']%2==1]
# evening df
edf = adf[adf['sessionID_x']%2==0]
edf_sin_nan = edf.copy()
edf_sin_nan['sleep']=list(mdf['sleep'])

key_SR = ['mood','food','sleep', 'real_stress']
key_PV = ['Dsubj_optout_oo','Ssubj_optout_oo','subj_perf_no','OKubj_RT_no','DoverConf','SoverConf','risk_av']
key_LR = ['slope_RT', 'slope_DO', 'slope_SO', 'intercept_RT', 'intercept_DO','intercept_SO']
key_name = key_SR+key_PV

key_SR2plot = [key_SR[:int(len(key_SR)/2)],key_SR[int(len(key_SR)/2):]]
pair_keys_SR = list(itertools.combinations(key_SR, 2))

# pair of PV 
pair_keys_PV = list(itertools.combinations(key_PV[:-3], 2))
pair_keys_PV2plot = [pair_keys_PV[:int(len(pair_keys_PV)/2)],pair_keys_PV[int(len(pair_keys_PV)/2):]]
key_name_PV = {'Dsubj_optout_oo':'DO','Ssubj_optout_oo':'SO','subj_perf_no':'acc NO','OKubj_RT_no':'RT NO'}
pair_key_name_PV2plot = [[[key_name_PV[k] for k in tupl] for tupl in lista] for lista in pair_keys_PV2plot]

SR_color = {'mood': 'b','food': 'r','sleep': 'y', 'real_stress':'g'}

key_diff = ['Dperf_oo', 'Sperf_oo', 'perf_no', 'Dn_trials_oo', 'Sn_trials_oo', 'NOn_trials', 'DRT_oo', 'SRT_oo', \
            'DRT_OKoo', 'SRT_OKoo', 'RT_no', 'RT_noNOK', 'Doptout', 'Soptout']


In [ ]:
adf.columns

In [ ]:
no_bias = adf['no_bias']
no_bias = no_bias[~np.isnan(no_bias)]

In [ ]:
pos_no_bias, neg_no_bias = [x for x in no_bias if x > 0],[x for x in no_bias if x <= 0]

In [ ]:
len(neg_no_bias)

In [ ]:
stats.ttest_1samp(no_bias,0)

In [ ]:
plt.hist(no_bias,bins=100)
plt.axvline(np.mean(no_bias),color='k')
plt.xlim(-0.25,0.25)
plt.show()

In [ ]:
t_value, p_value = [],[]
for uid in adf['userID'].unique():
    subset = adf[adf['userID']==uid]
    x = subset['no_bias']
    x = x[~np.isnan(x)] 
    test = stats.ttest_1samp(x,0)
    t_value.append(test[0])
    p_value.append(test[1])
    print(uid,test)

In [ ]:
plt.hist(t_value)
plt.axvline(np.mean(t_value),color='k')
plt.show()

In [ ]:
plt.hist(p_value,bins=100)
plt.axvline(0.05,color='k')
plt.show()